In [1]:
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
from sklearn.metrics import f1_score

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
!pip install import_ipynb
import import_ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, load_model, save_model
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, Masking, Bidirectional,Conv1D, MaxPooling1D
import tensorflow as tf
from sklearn.preprocessing import StandardScaler



<h1> normal data</h1>

In [4]:
from google.colab import drive
drive.mount('/content/drive')#구글 드라이브와 mount

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

normal_path = "/content/drive/MyDrive/information_system/normal/"
malicious_path = "/content/drive/MyDrive/information_system/malicious/"

normal=[]
for i in range(5):
    normal.append(pd.read_csv(normal_path+'network_benign_training_set_'+str(i).zfill(2)+'.csv',error_bad_lines=False))
attack=pd.read_csv(malicious_path+'network_malicious_training_set.csv',error_bad_lines=False)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (12,13,14,15,16,18,19,21) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Time 속성 전처리

In [7]:
import datetime
for i in range(5):
  normal[i]['_ws.col.UTCtime']=pd.to_datetime(normal[i]['_ws.col.UTCtime'])
  normal[i]['_ws.col.UTCtime']=normal[i]['_ws.col.UTCtime'].astype(np.int64)//10**9
  normal[i]['_ws.col.UTCtime']=normal[i]['_ws.col.UTCtime']-normal[i].iloc[0,0]
  normal[i]=normal[i].loc[:,"_ws.col.UTCtime":"tcp.ack"]
  normal[i]=normal[i].dropna(axis=0)
  normal[i]
  normal[i].to_csv("normal"+str(i)+".csv",mode='w',index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  after removing the cwd from sys.path.


In [8]:
attack['_ws.col.UTCtime']=pd.to_datetime(attack['_ws.col.UTCtime'])
attack['_ws.col.UTCtime']=attack['_ws.col.UTCtime'].astype(np.int64)//10**9
attack['_ws.col.UTCtime']=attack['_ws.col.UTCtime']-attack.iloc[0,0]
attack=attack.loc[:,"_ws.col.UTCtime":"tcp.ack"]
attack=attack.dropna(axis=0)
attack
attack.to_csv("attack"+str(i)+".csv",mode='w',index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  


In [9]:
for i in range(len(normal)):
    normal[i]=normal[i].astype({"_ws.col.UTCtime":'int','tcp.srcport': 'int','tcp.dstport': 'int','tcp.len': 'int','tcp.seq':'int','tcp.ack': 'int'})
attack=attack.astype({"_ws.col.UTCtime":'int','tcp.srcport': 'int','tcp.dstport': 'int','tcp.len': 'int','tcp.seq':'int','tcp.ack': 'int'})

In [10]:
#normal data를 그룹핑하고 100개가 넘는 그룹을 골라내기
groupset=[]
over_100_group=[]
for i in range(len(normal)):
    gb = normal[i].groupby(['_ws.col.Protocol','ip.src','ip.dst','tcp.srcport'])
    for key, group in gb:
        group = np.asarray(group)
        if len(group)>100:
            over_100_group.append(group)
        else :
            groupset.append(group)

In [11]:
for i in range(len(normal)):
    gb = normal[i].groupby(['_ws.col.Protocol','ip.src','ip.dst','tcp.dstport'])
    for key, group in gb:
        group = np.asarray(group)
        if len(group)>100:
            over_100_group.append(group)
        else :
            groupset.append(group)

In [12]:
#normal data 100개가 넘는 그룹 100개씩 잘라서 그룹셋에 넣기
for i in range(len(over_100_group)):
    for j in range(0,len(over_100_group[i]),100):
        groupset.append(over_100_group[i][j:j+100])

In [13]:
len(groupset)

154492

In [14]:
for i in range(len(groupset)):
    groupset[i][:,0]=groupset[i][:,0]-groupset[i][0][0]

In [15]:
#100개가 넘는것은 없는지 확인
for i in range(len(groupset)):
    if len(groupset[i])>100:
        print(i)

In [16]:
X=[]
for i in range(len(groupset)):
    temp=np.delete(groupset[i],[1,2,3,4,5],1)
    num=100-len(temp)
    X.append(np.pad(temp,((0,num),(0,0)),'constant', constant_values=-1))

In [17]:
X_data=np.asarray(X)

In [18]:
len(X_data)

154492

In [19]:
Y_data=[]
for i in range(len(X_data)):
    Y_data.append(0)

In [20]:
len(Y_data)

154492

In [21]:
X_data.shape

(154492, 100, 4)

공격데이터

In [22]:
groupset_a=[]
over_100_group_a=[]
gb = attack.groupby(['_ws.col.Protocol','ip.src','ip.dst','tcp.srcport'])
for key, group in gb:
    group = np.asarray(group)
    if len(group)>100:
        over_100_group_a.append(group)
    else :
        groupset_a.append(group)


In [23]:
gb = attack.groupby(['_ws.col.Protocol','ip.src','ip.dst','tcp.dstport'])
for key, group in gb:
    group = np.asarray(group)
    if len(group)>100:
        over_100_group_a.append(group)
    else :
        groupset_a.append(group)


In [24]:
for i in range(len(over_100_group_a)):
    for j in range(0,len(over_100_group_a[i]),100):
        groupset_a.append(over_100_group_a[i][j:j+100])

In [25]:
for i in range(len(groupset_a)):
    groupset_a[i][:,0]=groupset_a[i][:,0]-groupset_a[i][0][0]

In [26]:
X_a=[]
for i in range(len(groupset_a)):
    temp=np.delete(groupset_a[i],[1,2,3,4,5],1)
    num=100-len(temp)
    X_a.append(np.pad(temp,((0,num),(0,0)),'constant', constant_values=-1))

In [27]:
Y_a=[]
for i in range(len(X_a)):
    Y_a.append(1)

In [28]:
X_attack_data=np.asarray(X_a)
Y_attack_data=np.asarray(Y_a)

In [29]:
X_total=np.concatenate((X_data,X_attack_data), axis=0)
#index : 0~891936 까지 normal, 총 891937

In [30]:
X_total.shape

(154851, 100, 4)

In [31]:
Y_total=np.concatenate((Y_data,Y_attack_data), axis=0)

In [32]:
Y_total.shape

(154851,)

In [33]:
# Conv1d 모델을 이용해 특징 맵 추출 후 가중치 부여, LSTM 단일 은닉층을 배치.

def build_model():
    learning_rate = 0.00001
    seq_length = 100
    data_dim = 4
    METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy')
    ]
    model = Sequential()

    model.add(Conv1D(256, 3, activation = 'relu', kernel_regularizer='l2', input_shape=(100, 4)))
    model.add(MaxPooling1D(pool_size=4))    
    model.add(Dense(128, activation='relu', kernel_regularizer='l2'))
    model.add(LSTM(256))  
    model.add(Dense(128, activation='relu', kernel_regularizer='l2'))
    model.add(Dense(1, activation='sigmoid', kernel_regularizer='l2'))

    model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=learning_rate), metrics=METRICS)
    
    return model

In [34]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [35]:
skf = StratifiedKFold(n_splits=10)
k=10

In [39]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE

In [ ]:
all_history=[]
all_acc=[]
all_f1=[]
matrix=[]

model =  build_model()
for train_index, test_index in skf.split(X_total,Y_total):
    #print('train_index : ', train_index,'\ntest_index : ',test_index)
    X_train = X_total[train_index]
    y_train = Y_total[train_index]  
    X_test = X_total[test_index]
    y_test = Y_total[test_index]

    sm = SMOTE()
    X_train=np.reshape(X_train,(len(X_train),400))

    X_train_oversampled, y_train_oversampled = sm.fit_resample(X_train, y_train)
    X_train_oversampled=np.reshape(X_train_oversampled,(len(X_train_oversampled),100,4))
    history =model.fit(X_train_oversampled, y_train_oversampled,epochs=60, batch_size=256, verbose=1)
    all_history.append(history.history)
    X_test=X_test.astype('float32')
    y_pred = model.predict(X_test)
    y_pred_binary=np.around(y_pred)
    
    acc=accuracy_score(y_test, y_pred_binary)
    all_acc.append(acc)
    
    f1=f1_score(y_test, y_pred_binary)
    all_f1.append(f1)
    
    matrix.append(confusion_matrix(y_test, y_pred_binary))
    
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_binary).ravel()
    print(f'Accuracy: {acc}')
    print(f'f-score: {f1}')
    print(f'tn fp fn tp :{tn, fp, fn, tp}')

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/60
 839/1087 [======================>.......] - ETA: 2:02 - loss: 3.5612 - accuracy: 0.9190

In [ ]:
print(all_acc)
print(all_f1)
for i in range(k):
    print(matrix[i][0][0],matrix[i][0][1],matrix[i][1][0],matrix[i][1][1])

In [ ]:
print(sum(all_acc)/10)
print(sum(all_f1)/10)
for i in range(k):
    print(matrix[i][0][0],matrix[i][0][1],matrix[i][1][0],matrix[i][1][1])

In [ ]:
for i in range(5):
    loss =all_history[i]['loss']
    #val_loss = all_history[i]['val_loss']
    epochs = range(1, len(loss) + 1)
    plt.plot(epochs, loss, 'bo', label='Training loss')  # ‘bo’는 파란색 점을 의미합니다.
    #plt.plot(epochs, val_loss, 'b', label='Validation loss') # ‘b’는 파란색 실선을 의미합니다.
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()